In [1]:
#-----------------------------------------------------------------------------------------------------------------
#importing libraries
from __future__ import print_function, division
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.interpolate as interp
import argparse
import copy
import datetime
import math
from matplotlib.gridspec import GridSpec
global low, up
from astropy.io import fits as f
from astropy.table import Table as t
from astropy.utils.data import download_file
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
from mpl_toolkits import mplot3d
from scipy.optimize import curve_fit
import scipy.optimize as so
import warnings
warnings.filterwarnings("ignore")
#-----------------------------------------------------------------------------------------------------------------

In [5]:
#importing data from my GitHub
# url = 'https://raw.githubusercontent.com/jovian-explorer/Sun/main/MOM/output_expdopp_4point.csv'
url = '/home/dev/Sun/test.txt'
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_colwidth', None)
dataset = pd.read_csv(url,delimiter = '\t')
print(dataset.head(1))
#-----------------------------------------------------------------------------------------------------------------

   ET_time_sec    time_sec_of_day    freq_residual    exp_dopp    theo_dopp 
0  6.865017e+08            10405.0     -2905.642025   8403.3824  11309.02442


In [7]:
#-----------------------------------------------------------------------------------------------------------------
# accessing the different columns of the csv dataset I made above
dataset.columns = [   'ET_time_sec',  'time_sec_of_day',    'freq_residual',    'exp_dopp',    'theo_dopp' ]
#storing values for each column into a separate arrays

'''
Finding Moments of powSpec (Power spectral density distribution)- by defining "powSpecmoment" function - 
zeroth moment = power, 
first moment = doppler freq
second = width of freq spread 
downconfreq - RF_TO_IF downconv frequency as read from header file
final experimental/Observed doppler shift observed in received signal is (given by heterodyning principle)  
exp_dopp = Downconversion frequency + dopp_firstmoment_freq - transmitted freq
IV - write output in "output.txt" file containing :  time in seconds----power(first moment)-----measured dopp freq
'''
ET_time_sec= np.array(dataset.ET_time_sec)              
time_sec_of_day= np.array(dataset.time_sec_of_day)                
freq_residual= np.array(dataset.freq_residual)                  # Values of freq_residual
exp_dopp= np.array(dataset.exp_dopp)                	        # Values of exp_dopp
theo_dopp = np.array(dataset.theo_dopp)	                        # Values of theo_dopp
#-----------------------------------------------------------------------------------------------------------------

In [10]:
#------------------------------------------------
m = 9.1*10**(-31)
e = 1.6*10**(-19)
frec = 8410640000.0
k= 5.9
l_2= 2.4235* 10**11
alpha = 1.14*10**(-24)
L = 3.9195*10**11
ds=(1.469* 10**11)/len(ET_time_sec)
R = 1.252* 10**10
downconfreq = 8410640000.0
#------------------------------------------------

In [13]:
RTEC = (freq_residual/(alpha*(downconfreq)**(-1.2)))**(1/1.2)

density = RTEC/R
plasma_frequency_model_1 = np.sqrt(4*np.pi*density*(e**2)/m)#Yamato et al. 2002
plasma_frequency_model_2 = np.sqrt(density*(e**2)/(np.pi*m))#Chen F. F., 1974

#refractive index equation taken from Hollweg et al. 1970
refractive_index_model_1 = np.sqrt(1 - (plasma_frequency_model_1/frec)**2)
refractive_index_model_2 = np.sqrt(1 - (plasma_frequency_model_2/frec)**2)

refractive_index_model_1 = np.nan_to_num(refractive_index_model_1)
refractive_index_model_2 = np.nan_to_num(refractive_index_model_2)

d_refractive_index_model_1 = np.gradient(refractive_index_model_1,1)
d_refractive_index_model_2 = np.gradient(refractive_index_model_2,1)
dy = np.gradient(freq_residual)

#angular broadening taken from Hollweg et al. 1970

temp_1 = d_refractive_index_model_1/dy
temp_2 = d_refractive_index_model_2/dy
temp_1 = temp_1*ds
temp_2 = temp_2*ds

angular_broadening_model_1 = np.sum(temp_1)
angular_broadening_model_2 = np.sum(temp_2)

#Solar wind velocity taken from Bird et al. 1982
solar_wind_velocity_model_1 = (5.77*frec*L)/(angular_broadening_model_1*k*l_2) 
solar_wind_velocity_model_2 = (5.77*frec*L)/(angular_broadening_model_2*k*l_2)

In [14]:
print(temp_1)
print(temp_2)

print(angular_broadening_model_1)
print(angular_broadening_model_2)

print(solar_wind_velocity_model_1)
print(solar_wind_velocity_model_2)

[-0.  0.  0. ...  0. -0. -0.]
[-0.  0.  0. ...  0. -0. -0.]
619049.4748806442
619080.6888744546
21488.94681487721
21487.863344071346
